In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as ppt
from sklearn import linear_model
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from linearmodels import OLS
from linearmodels import PanelOLS
from linearmodels import PooledOLS, IV2SLS
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

More formally, a variable z is called an instrument or instrumental variable for the regressor x in the scalar regression model y = x+u if (1) z is uncorrelated with the error u;and (2) z is correlated with the regressor x.

http://cameron.econ.ucdavis.edu/e240a/ch04iv.pdf

In [2]:
# Create data
a = 1.
b = 2.
c = 3.
d = 4.
sigma = 1.
df_panel = pd.DataFrame(index=range(1000))
df_panel['u'] = np.random.uniform(0, 10, df_panel.shape[0])
df_panel['z'] = np.random.uniform(0, 10, df_panel.shape[0])
df_panel['eps_1'] = np.random.normal(0, sigma, df_panel.shape[0])
df_panel['x'] = a*df_panel['u'] + b*df_panel['z'] + df_panel['eps_1']
df_panel['eps_2'] = np.random.normal(0, sigma, df_panel.shape[0])
df_panel['y'] = c*df_panel['u'] + d*df_panel['x'] + df_panel['eps_2']
df_panel['const'] = 1.
df_panel.head()

,u,z,eps_1,x,eps_2,y,const
0,3.239249,0.188221,0.807058,4.422750,1.134344,28.543092,1.0
1,1.674966,4.056534,-0.623976,9.164059,-0.336001,41.345131,1.0
2,1.759627,0.644776,-1.658941,1.390239,0.354114,11.193950,1.0
3,1.021257,8.053543,-0.681322,16.447021,-0.097981,68.753876,1.0
4,5.954716,9.260877,-0.073081,24.403389,-1.177226,114.300479,1.0


## Without IV: over-estimates coefficient of x

In [9]:
# With no instrument
dep = df_panel['y']
exog = df_panel[['const','x']]
mod = OLS(dep, exog) #, entity_effects=True,time_effects=True)
res = mod.fit() #cov_type='clustered', cluster_entity=True)
print(res)

                            OLS Estimation Summary                            
Dep. Variable:                      y   R-squared:                      0.9376
Estimator:                        OLS   Adj. R-squared:                 0.9375
No. Observations:                1000   F-statistic:                 2.393e+04
Date:                Fri, Jul 09 2021   P-value (F-stat)                0.0000
Time:                        09:57:37   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          6.0986     0.5116     11.920     0.0000      5.0959      7.1014
x              4.6241     0.0299     154.68     0.00

In [10]:
## Alternatively : Two step regression without instruments --> Leads to the same result
dep = df_panel['y']
exog = df_panel[['const','x']]
mod = IV2SLS(dep, exog, None, None) #, entity_effects=True,time_effects=True)
res = mod.fit() #cov_type='clustered', cluster_entity=True)
print(res)

                            OLS Estimation Summary                            
Dep. Variable:                      y   R-squared:                      0.9376
Estimator:                        OLS   Adj. R-squared:                 0.9375
No. Observations:                1000   F-statistic:                 2.393e+04
Date:                Fri, Jul 09 2021   P-value (F-stat)                0.0000
Time:                        09:57:51   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          6.0986     0.5116     11.920     0.0000      5.0959      7.1014
x              4.6241     0.0299     154.68     0.00

## With IV: manual two-step regression

In [11]:
# With instrument
exog = df_panel[['const','z']]
endog = df_panel['y']
mod = OLS(endog, exog) #, entity_effects=True,time_effects=True)
res = mod.fit() #cov_type='clustered', cluster_entity=True)
print(res)

                            OLS Estimation Summary                            
Dep. Variable:                      y   R-squared:                      0.5485
Estimator:                        OLS   Adj. R-squared:                 0.5480
No. Observations:                1000   F-statistic:                    1218.4
Date:                Fri, Jul 09 2021   P-value (F-stat)                0.0000
Time:                        09:58:30   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          35.932     1.3192     27.239     0.0000      33.347      38.518
z              8.0007     0.2292     34.906     0.00

In [12]:
exog = df_panel[['const','z']]
endog = df_panel['x']
mod = OLS(endog, exog) #, entity_effects=True,time_effects=True)
res = mod.fit() #cov_type='clustered', cluster_entity=True)
print(res)

                            OLS Estimation Summary                            
Dep. Variable:                      x   R-squared:                      0.7728
Estimator:                        OLS   Adj. R-squared:                 0.7725
No. Observations:                1000   F-statistic:                    3377.7
Date:                Fri, Jul 09 2021   P-value (F-stat)                0.0000
Time:                        09:58:30   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          5.1962     0.1976     26.298     0.0000      4.8089      5.5835
z              1.9886     0.0342     58.118     0.00

Result is given by dividing coefficient of z on y by the coefficient of z on x

In [13]:
8.0007/1.9886

4.023282711455296

## With IV: using linearmodels' IV2SLS

In [16]:
## Two step regression
mod = IV2SLS(df_panel.y, df_panel.const, df_panel.x, df_panel.z) #, entity_effects=True,time_effects=True)
res = mod.fit() #cov_type='clustered', cluster_entity=True)
print(res)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                      y   R-squared:                      0.9217
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9217
No. Observations:                1000   F-statistic:                    7093.1
Date:                Fri, Jul 09 2021   P-value (F-stat)                0.0000
Time:                        10:00:46   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          15.027     0.7680     19.567     0.0000      13.522      16.532
x              4.0232     0.0478     84.221     0.00